In [8]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using RandomizedLinAlg
using RData  # Load the RData package
using RCall

R"""
library(GPvecchia) # i had to install this package i added this 
"""

RObject{StrSxp}
[1] "GPvecchia" "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     


In [10]:
include("../../utils/util2.j")
include("../../utils/precalc_prealloc_function.jl") ## adds the file with preallocation and precalculation function

initialization (generic function with 1 method)

In [11]:
@load "../data/sim1.jld";

In [12]:
## can delete later, just showing what's in the jld file
file = jldopen("../data/sim1.jld", "r")

# List all variable names in the file
keys(file)

10-element Vector{String}:
 "Y"
 "X"
 "coords"
 "n"
 "q"
 "p"
 "F0"
 "β"
 "Λ"
 "sqrt_Σ_diag"

In [13]:
# defining variables here that weren't defined in jld file for use in pre calc/pre alloc function (so we can easily try other stuff)
ϕ_sam=[4.0, 6.0]
K=2

2

In [14]:
## Precalculation and Preallocation ## 
pre_calc_alloc =  initialization(p,q,n,coords,X,Y,ϕ_sam,K;m=10, N_sam = 20000)

(Y_ord = [-2.94271092003418 1.6457308823905574 … 3.6367612621139624 0.1769901839563932; -1.1875966058415433 0.8268824695018478 … 1.8564406486314542 -0.5852152175830511; … ; 0.02655299199157435 -0.08086755804098922 … -0.6338418497228471 0.8410272195003013; 1.1495883587866431 0.571907121626949 … -4.0336616139305415 0.03505506265347402], X_ord = [1.0 0.870695399280016; 1.0 0.7039441456578376; … ; 1.0 0.33390780843127843; 1.0 0.15343656880275625], coords_ord = [0.5040517961245712 0.4920734135317657; 0.9987386947284053 0.9989295299172429; … ; 0.789173845424874 0.47751409824924373; 0.7795910567770608 0.7719668669260211], F_ord = [-1.7386231449256888 -0.21543790073546049; -0.45730745227298286 0.5058537407531963; … ; 0.6915391392632595 -1.0051114941724728; -0.3696838260220323 0.47965708513448835], NN = (nnIndx = [1, 1, 2, 1, 2, 3, 1, 3, 4, 2  …  420, 1326, 1406, 1873, 852, 1992, 1264, 643, 1271, 886], nnDist = [0.7082501326280487, 0.6995740543556118, 0.9928152728787168, 0.6920743695673944, 0.9

In [15]:
# delete once function is confirmed
## Precalculation I ##
# Construct maximin ordering
# Convert Julia matrix to R
@rput coords
# Use order_maxmin_exact from GPvecchia
R"""
library(GPvecchia) # i had to install this package i added this 
# Apply maxmin ordering using the built-in function
ordered_indices <- order_maxmin_exact(coords)
"""
# Get results back to Julia
@rget ordered_indices;

In [16]:
# delete once function is confirmed
# reorder data #
Y_ord = Y[ordered_indices, :];
X_ord = X[ordered_indices, :];
coords_ord = coords[ordered_indices, :];
F_ord = F0[ordered_indices, :]; # not needed here, used in model comparison step

In [ ]:
# delete once function is confirmed
# build nearest neighbor, might take some time 
m = 15
K = 2 # No. factors
NN = BuildNN(coords_ord, m);

In [ ]:
# delete once function is confirmed
## Precalculation II ##
# priors #
K = 2;
μβ = fill(0.0, p, q); inv_Vr = zeros(Float64, p, p); #inv_Vr[1, 1] = 1;
μΛ = fill(0.0, K, q); inv_VΛ = zeros(Float64, K, K);
aΣ = 2; bΣ = fill(1, q); # later will want to add something to estimate sigma//initial starting point, here we jsut assumed something 
inv_Lr = sparse(1:p, 1:p, fill(0.1, p));
inv_LΛ = sparse(1:K, 1:K, fill(0.1, K));
ϕ_sam = [4.0, 6.0]; #[6.0, 9.0] #practical range from 3000 to 150

# A and D #
nIndx = length(NN.nnIndx)
A = [Array{Float64}(undef, nIndx) for i in 1:K]
D = [Array{Float64}(undef, n) for i in 1:K]
I_A = [spzeros(n, n) for i in 1:K] # 1-A
A_new = [Array{Float64}(undef, nIndx) for i in 1:K]
D_new = [Array{Float64}(undef, n) for i in 1:K]
I_A_new = [spzeros(n, n) for i in 1:K]
    
nnIndx_col = vcat(NN.nnIndx, 1:n) # Index of columns (for getting the original nxn matrix back; we stored it us spzeros so it takes less space)
nnIndx_row = zeros(Int64, 0) # Index of rows initialization
for i in 2:m
    nnIndx_row = vcat(nnIndx_row, fill(i, i-1))
end
nnIndx_row = vcat(nnIndx_row, repeat((m + 1):n, inner = m), 1:n)
dim_invD = n*q;

In [ ]:
# delete once function is confirmed
## Pre-allocation ##
# Pre-allocation for MCMC samples
N_sam = 20000;

# Pre-allocation for F updates
nsam = (n * q) + (K * n);
Xtilde = spzeros(Float64, (q+K)*n, n*K);
Ytilde = Array{Float64}(undef, nsam);
F_sam = Array{Float64, 2}(undef, n, K);
Fqr = qr(zeros(Float64, n, K)) 
inv_sqrt_Σ_diag = Vector{Float64}(undef, q)
invD_ele = Vector{Float64}(undef, n*q)
invD = spzeros(Float64, n*q, n*q)
invΣhalf = spzeros(Float64, q, q)
F_m = zeros(Float64, 1, K);
v = Array{Float64}(undef, nsam); # storage samples from standard normal

# Pre-allocation for γ, Σ updates
Ystar = vcat(Y_ord, inv_Lr * μβ, fill(0.0, K, q)); # (NTotal+p+K) by q matrix
Xstar = vcat(hcat(X_ord, F_sam), hcat(inv_Lr, spzeros(p, K)), hcat(spzeros(K, p), sparse(1:K, 1:K, 1.0)));
μγstar = vcat(μβ, μΛ); #invVγstar = fill(0.0, p + K, p + K);
invVγstar = cholesky(sparse(I, p+K, p+K)); # doesn't fine in-place update for this 
u = Array{Float64}(undef, (p + K) * q);  # Pre-allocate space for random samples;
Y_Xm = spzeros(n + p + K, q); # store the residual
bstar = fill(0.0, q); astar = aΣ + 0.5 * (n);

In [7]:
## Initalization (some are optional) ##
β0 = (X_ord'X_ord)\(X_ord'Y_ord);
Residuals = Y_ord - X_ord*β0;
#γ_sam = vcat((X'X)\(X'Y), fill(0.0, K, q));
reordered_result = reorder_svd_by_spatial_range(Residuals, coords_ord, K, 1000, 11)
γ_sam = vcat((X_ord'X_ord)\(X_ord'Y_ord), reordered_result);
Σ_sam = [var(Residuals[:, j]) for j in 1:size(Residuals, 2)];

# force the initial column of F to start from the noisy factor 
#γ_sam = vcat((X'X)\(X'Y), Λ[[2, 1], :]);
#Σ_sam = [0.5, 1, 0.4, 2, 0.3, 2.5, 3.5, 0.45, 1.5, 0.5]; #fill(1.0, q);

LoadError: UndefVarError: `X_ord` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
using DelimitedFiles
writedlm("../results/PMCMC/γ_sam.csv", fill(0.0, 1, q), ", ");
writedlm("../results/PMCMC/Σ_sam.csv", 0.0, ", ");
writedlm("../results/PMCMC/F_sam.csv", fill(0.0, 1, K), ", ");

In [ ]:
# for loop for MCMC chain #
Random.seed!(11);
prog = Progress(N_sam, 1, "Computing initial pass...", 50)
for l in 1:N_sam
    # Build the matrix D_Sigma_o^{1/2} #
    # Compute inverse square root in-place
    @. inv_sqrt_Σ_diag = 1 / sqrt(Σ_sam)
    # Efficient broadcasting for invD_ele
    invD_ele .= repeat(inv_sqrt_Σ_diag, inner=n)
    # Update sparse matrices
    invD = sparse(1:(n*q), 1:(n*q), invD_ele)
    invΣhalf = sparse(1:q, 1:q, 1 ./ sqrt.(Σ_sam))
    
    if l == 1
        for k in 1:K
            getAD(coords_ord, NN.nnIndx, NN.nnDist, NN.nnIndxLU, ϕ_sam[k, l], 0.5, A[k], D[k]);
            I_A[k] = sparse(nnIndx_row, nnIndx_col, vcat(-A[k], ones(n)));
        end
    end
    Ytilde .= vcat(invD * vec(Y_ord - X_ord * γ_sam[1:p, :]), zeros(K * n));
    Xtilde .= vcat(kron(invΣhalf * sparse(transpose(γ_sam[(p + 1):(p + K), :])), 
                            sparse(1:n, 1:n, ones(n))),
             blockdiag([Diagonal(1 ./ sqrt.(D[k])) * I_A[k] for k in 1:K]...));
       
    # use LSMR to generate sample of F # 
    randn!(v)  # Fills v with standard normal samples
    elapsed_time = @elapsed begin
    F_sam .= reshape(lsmr(Xtilde, collect(Ytilde) + v), :, K);
    end
    
    # Print the elapsed time
    #println("Iteration $l: Time taken for lsmr step = $elapsed_time seconds")
    
    F_m .= mean(F_sam, dims = 1);
    F_sam .-= F_m;

    # Perform thin QR on the tall slice of F_sam in-place
    Fqr = qr!(F_sam)  # Note the ! for in-place modification

    # Assign scaled Q to F_samples
    F_sam .= Matrix(Fqr.Q)
    rmul!(F_sam, sqrt(n-1))   # scale in-place
    
    io1 = open("../results/PMCMC/F_sam.csv", "a" );
    writedlm(io1, F_sam, ", ");
    close(io1);

    # Sample Σ and γ#
    Xstar[1:n, (p+1):(p+K)] .= F_sam; # update Xstar with F_sam
    
    # use MNIW to sample γ Σ #
    invVγstar = cholesky(Xstar'Xstar);
    #mul!(μγstar, transpose(Xstar), Ystar); μγstar = invVγstar \ μγstar;
    μγstar .= invVγstar \ (Xstar'Ystar);
    Y_Xm .= Ystar - Xstar * μγstar;      
    bstar .= [bΣ[ind] + 0.5 * (norm(Y_Xm[:, ind])^2) for ind in 1:q];
    Σ_sam .= [rand(InverseGamma(astar, bstar[ind]), 1)[1] for ind in 1:q];          # sample Σ
    randn!(u)  # Fills u with standard normal samples
    γ_sam .= (invVγstar.U \ reshape(u, (p + K), q)) * 
                    Diagonal(sqrt.(Σ_sam)) + μγstar;          # sample γ    

    
    io2 = open("../results/PMCMC/Σ_sam.csv", "a" );
    writedlm(io2, Σ_sam, ", ");
    close(io2);
    io3 = open("../results/PMCMC/γ_sam.csv", "a" );
    writedlm(io3, γ_sam, ", ");
    close(io3)
    
    next!(prog) # monitor the progress
end

In [ ]:
#load data
using CSV
using DataFrames
using StatsBase

γ_sam = Matrix{Float64}(CSV.read("../results/PMCMC/γ_sam.csv", DataFrame));
ind_γ_sam = (1): (p + K) :((p + K) * N_sam);
Σ_sam = Matrix{Float64}(CSV.read("../results/PMCMC/Σ_sam.csv", DataFrame));
ind_Σ_sam = (1): q :(q * N_sam);

In [ ]:
F_sam = Matrix{Float64}(CSV.read("../results/PMCMC/F_sam.csv", DataFrame));
ind_F_sam = (1): n :(n * N_sam);

In [ ]:
N_warmup = Integer(trunc(0.25 * N_sam));

In [ ]:
# post-processing #
# step 1 compute the posterior mean
Λ_mean = Array{Float64, 2}(undef, K, q);
for k in 1:K
    Λ_mean[k, :] = mean(γ_sam[((k + p)+ N_warmup*(p+K)):(p+K):(k+p+(p+K)*(N_sam-1)), :], dims = 1);
end
Λ_mean

# step 2 label switching
labels = zeros(Int, N_sam, K);

# For each iteration and each k
for i in 1:N_sam
    for k in 1:K
        # Extract current row sample and corresponding mean row
        row_sample = γ_sam[((k + p)+ (i-1)*(p+K)), :];
        row_mean = Λ_mean[k,:];
        
        # Compute inner product and determine sign
        inner_prod = dot(row_sample, row_mean)
        labels[i,k] = sign(inner_prod)
        
        # Switch the sign when the inner product is negative
        if labels[i,k] == -1 
            γ_sam[((k + p)+ (i-1)*(p+K)), :] = -γ_sam[((k + p)+ (i-1)*(p+K)), :];
            F_sam[(1+(i-1)*n):((i)*n), k] = -F_sam[(1+(i-1)*n):((i)*n), k]; # flip sign for iteration i for F
        end
    end
end
[countmap(labels[(N_warmup+1):N_sam, col]) for col in 1:K]

In [ ]:
#=
# post-processing for F #
# step 1 compute the posterior mean
F_mean = [mean(F_sam[(i .+ (n*(N_warmup+1)):n:(n*N_sam)), j]) 
    for i in 1:n, j in 1:K];

# step 2 label switching
labels_F = zeros(Int, N_sam, K);

# For each iteration and each k
for i in 1:N_sam
    for k in 1:K
        # Extract current row sample and corresponding mean row
        f_sample = F_sam[(1+(i-1)*n):((i)*n), k];
        f_mean = F_mean[:, k];
        
        # Compute inner product and determine sign
        inner_prod = dot(f_sample, f_mean)
        labels_F[i,k] = sign(inner_prod)
        
        # Switch the sign when the inner product is negative
        if labels_F[i,k] == -1 
            F_sam[(1+(i-1)*n):((i)*n), k] = -F_sam[(1+(i-1)*n):((i)*n), k]; 
        end
    end
end
[countmap(labels_F[(N_warmup+1):N_sam, col]) for col in 1:K]
=#

In [ ]:
#[countmap((labels_F - labels)[:, col]) for col in 1:K]

# MCMC Chain check

In [ ]:
# packages for diagnostic test
using StatsPlots
using MCMCChains
using PrettyTables
using MCMCDiagnosticTools

In [ ]:
β_pos_sam = Array{Float64, 3}(undef, N_sam , p * q, 1);
β_pos_sam[:, :, 1] = hcat([γ_sam[ind_γ_sam .+ (i - 1), j] for i in 1:p, j in 1:q]...);
β0_chain = Chains(β_pos_sam[(N_warmup+1):N_sam, 1:2:(q*p), :]);
#pβ0 = plot(β0_chain) # check as needed

In [ ]:
# For β1 (remaining coefficients) - taking all columns except the intercepts
remaining_cols = setdiff(1:(p*q), 1:p:(q*p))  # Get all column indices except intercepts
β1_chain = Chains(β_pos_sam[(N_warmup+1):N_sam, remaining_cols, :]);

In [ ]:
Λ_pos_sam = Array{Float64, 3}(undef, N_sam, K * q, 1);
Λ_pos_sam[:, :, 1] = hcat([γ_sam[ind_γ_sam .+ (p + i - 1), j] for i in 1:K, j in 1:q]...)
Λ_chain = Chains(Λ_pos_sam[(N_warmup+1):N_sam, :, :]);#[:, [2, 8, 10, 12, 13, 15, 19, 22, 29, 33, 38, 41, 43], :]); #N_warmup
#pΛ = plot(Λ_chain) # check as needed

In [ ]:
F_pos_sam = Array{Float64, 3}(undef, N_sam, K*n, 1);
F_pos_sam[:, :, 1] = hcat([F_sam[ind_F_sam .+ (i - 1), j] for i in 1:n, j in 1:K]...)
F_chain = Chains(F_pos_sam[(N_warmup+1):N_sam, :, :]);#[:, [2, 8, 10, 12, 13, 15, 19, 22, 29, 33, 38, 41, 43], :]);
#pF = plot(F_chain) 

In [ ]:
#F_chain = Chains(F_pos_sam[(N_warmup+1):N_sam, [100, 208, 310, 412, 513, 601, 702, 1300, 1400, 1033, 1380, 1410, 1830], :]);#[:, [2, 8, 10, 12, 13, 15, 19, 22, 29, 33, 38, 41, 43], :]);
#pF = plot(F_chain) # check as needed

In [ ]:
Σ_pos_sam = Array{Float64, 3}(undef, N_sam, q, 1);
Σ_pos_sam[:, :, 1] = hcat([Σ_sam[ind_Σ_sam .+ (ind - 1)] for ind in 1:q]...);
Σ_chain = Chains(Σ_pos_sam[(N_warmup+1):N_sam, :, :]);
#pΣ = plot(Σ_chain) # check as needed

In [ ]:
#mean(Σ_pos_sam[(N_warmup+1):N_sam, :, :], dims = 1)[:]  #[0.5, 1, 0.4, 2, 0.3, 2.5, 3.5, 0.45, 1.5, 0.5]

In [ ]:
#sum(mean(Σ_pos_sam[(N_warmup+1):N_sam, :, :], dims = 1))

In [ ]:
#sum(std(Y, dims = 1).^2)

In [ ]:
# generate table of ESS
using DataFrames
using Statistics

function generate_ess_summary(β0_chain, β1_chain, Λ_chain, F_chain, Σ_chain; digits::Int = 0)
    # Compute ESS for each parameter group
    β0_ess = ess(β0_chain)
    β1_ess = ess(β1_chain)
    Λ_ess = ess(Λ_chain)
    F_ess = ess(F_chain)
    Σ_ess = ess(Σ_chain)

    # Create a DataFrame to store results
    ess_summary = DataFrame(
        Parameter = ["β0", "β1", "Λ", "F", "Σ"],
        Minimum_ESS = round.(
            [minimum(β0_ess[:, 2]), minimum(β1_ess[:, 2]), minimum(Λ_ess[:, 2]), minimum(F_ess[:, 2]),
                minimum(Σ_ess[:, 2])], 
            digits = digits
        ),
        Mean_ESS = round.(
            [mean(β0_ess[:, 2]), mean(β1_ess[:, 2]), mean(Λ_ess[:, 2]), mean(F_ess[:, 2]), mean(Σ_ess[:, 2])], 
            digits = digits
        ),
        Median_ESS = round.(
            [median(β0_ess[:, 2]), median(β1_ess[:, 2]), median(Λ_ess[:, 2]), median(F_ess[:, 2]), 
                median(Σ_ess[:, 2])], 
            digits = digits
        ),
        Percent_ESS_Below_100 = round.(
            [
                count(β0_ess[:, 2] .< 100) / length(β0_ess[:, 2]) * 100,
                count(β1_ess[:, 2] .< 100) / length(β1_ess[:, 2]) * 100,
                count(Λ_ess[:, 2] .< 100) / length(Λ_ess[:, 2]) * 100,
                count(F_ess[:, 2] .< 100) / length(F_ess[:, 2]) * 100,
                count(Σ_ess[:, 2] .< 100) / length(Σ_ess[:, 2]) * 100
            ], 
            digits = digits
        )
    )

    return ess_summary
end

ess_table = generate_ess_summary(β0_chain, β1_chain, Λ_chain, F_chain, Σ_chain)

# Display the table
display(ess_table)

In [ ]:
# Compute the posterior mean (spherical mean) of F
# Compute Euclidean mean and project onto sphere
using LinearAlgebra
F_mean_euclidean = [mean(F_sam[(i .+ (n*(N_warmup+1)):n:(n*N_sam)), j]) 
    for i in 1:n, j in 1:K];
F_mean_projected = mapslices(normalize, F_mean_euclidean, dims=1) * sqrt(n-1);

#=
# project the true number onto sphere
F_proj = F_ord - fill(1.0, n) * mean(F_ord, dims = 1);
F_proj = mapslices(normalize, F_proj, dims=1);
Fqr2 = qr(F_proj)  # Note the ! for in-place modification
# Assign scaled Q to F_samples
F_proj2 = Matrix(Fqr2.Q)* sqrt(n-1);
=#

In [ ]:
#F_proj2[:, 2] = -F_proj2[:, 2];

In [ ]:
# scatter plots
using Plots
plot(
    scatter(F_mean_projected[:, 1], F_ord[:, 1], title="Factor 1"),
    scatter(F_mean_projected[:, 2], F_ord[:, 2], title="Factor 2"),
    layout = (1, 2),  # 1 row, 2 columns
    size = (800, 400)  # Total figure size
)

In [ ]:
# save the results
using CSV, DataFrames
# Assuming 'matrix' is your N by K matrix
df_F_mean = DataFrame(F_mean_euclidean, :auto)
CSV.write("../results/PMCMC/F_mean_euclidean.csv", df_F_mean)  # Save DataFrame to CSV file

In [ ]:
comparison_df = generate_comparison_table(F_mean_euclidean, F_mean_projected, F_ord);

In [ ]:
#using Plots

# Coordinates
lon = coords_ord[:, 1]
lat = coords_ord[:, 2]

# Calculate the combined range for color scaling
combined_values = vcat(F_mean_projected[:, 1], F_ord[:, 1])
color_limits = (minimum(combined_values), maximum(combined_values))

# Create subplot for both factors
plot(
    # Factor 1 plot
    scatter(
        lon, lat, 
        zcolor = F_mean_projected[:, 1], 
        color = :RdBu, 
        markerstrokewidth = 0, 
        legend = false,
        markersize = 10.0, 
        xlabel = "Longitude", 
        ylabel = "Latitude", 
        title = "Projected Factor 1",
        yflip = true,
        colorbar = true,
        clims = color_limits
    ),
    
    # Factor 2 plot
    scatter(
        lon, lat, 
        zcolor = F_ord[:, 1], 
        color = :RdBu, 
        markerstrokewidth = 0, 
        legend = false,
        markersize = 10.0, 
        xlabel = "Longitude", 
        ylabel = "Latitude", 
        title = "True Projected Factor 1",
        yflip = true,
        colorbar = true,
        clims = color_limits
    ),
    
    # Layout options
    layout = (1, 2),  # 1 row, 2 columns
    size = (1200, 500)  # Total figure size
)

In [ ]:
#using Plots

# Coordinates
lon = coords_ord[:, 1]
lat = coords_ord[:, 2]


# Calculate the combined range for color scaling
combined_values = vcat(F_mean_projected[:, 2], F_ord[:, 2])
color_limits = (minimum(combined_values), maximum(combined_values))

plot(
    # Factor 1 plot
    scatter(
        lon, lat, 
        zcolor = F_mean_projected[:, 2], 
        color = :RdBu, 
        markerstrokewidth = 0, 
        legend = false,
        markersize = 10.0, 
        xlabel = "Longitude", 
        ylabel = "Latitude", 
        title = "Projected Factor 2",
        yflip = true,
        colorbar = true,
        clims = color_limits  # Add common color limits
    ),
    
    # Factor 2 plot
    scatter(
        lon, lat, 
        zcolor = F_ord[:, 2], 
        color = :RdBu, 
        markerstrokewidth = 0, 
        legend = false,
        markersize = 10.0, 
        xlabel = "Longitude", 
        ylabel = "Latitude", 
        title = "True Projected Factor 2",
        yflip = true,
        colorbar = true,
        clims = color_limits  # Add common color limits
    ),
    
    # Layout options
    layout = (1, 2),  # 1 row, 2 columns
    size = (1200, 500)  # Total figure size
)

# Plots for paper  

In [ ]:
# tract plots
β_chain = Chains(β_pos_sam[1:N_sam, 1:2:(q*p), :]);
Λ_chain = Chains(Λ_pos_sam[1:N_sam, :, :]);
# Plot only the trace plots
p1 = plot()
for i in 1:size(β_chain.value, 2)
    plot!(β_chain[:, i, :], label="", legend=false, size=(800, 500))
end
savefig(p1, "../results/PMCMC/PMCMC_intercept_trace.png")
    
# Plot only the trace plots
p2 = plot()
for i in 1:size(Λ_chain.value, 2)
    plot!(Λ_chain[:, i, :], label="", legend=false, size=(800, 500))
end
savefig(p2, "../results/PMCMC/PMCMC_Lambda_trace.png")

In [ ]:
# Coordinates
lon = coords_ord[:, 1]
lat = coords_ord[:, 2]

# Iterate through factors (assuming 2 factors)
for k in 1:2
    # Calculate the combined range for color scaling for this factor
    combined_values = vcat(F_mean_projected[:, k], F_ord[:, k])
    color_limits = (minimum(combined_values), maximum(combined_values))

    # Projected Factor Plot
    p1 = scatter(
        lon, lat, 
        zcolor = F_mean_projected[:, k], 
        color = :RdBu, 
        markerstrokewidth = 0, 
        legend = false,
        markersize = 10.0, 
        yflip = true,
        colorbar = false,
        clims = color_limits,
        size = (500, 500)
    )

    # Save projected factor plot
    Plots.savefig(p1, "../results/PMCMC/PMCMC_estimated_F$k.png")

    # True Factor Plot
    p2 = scatter(
        lon, lat, 
        zcolor = F_ord[:, k], 
        color = :RdBu, 
        markerstrokewidth = 0, 
        legend = false,
        markersize = 10.0, 
        yflip = true,
        colorbar = false,
        clims = color_limits,
        size = (500, 500)
    )

    # Save true factor plot
    Plots.savefig(p2, "../results/PMCMC/PMCMC_True_F$k.png")
end